## Demo: Mapping a hydrologic basin data with Folium

For this demo, I am plotting various features within a hydrologic basin in northern New Mexico. 

All of the data in the map is retrieved using the [`pynhd`](https://github.com/hyriver/pynhd) package from the [HyRiver](https://hyriver.readthedocs.io/en/latest/) suite for python. For more information about these packages, see my previous post, [*Efficient hydroclimatic data accessing with HyRiver for Python*.](https://waterprogramming.wordpress.com/2022/09/20/efficient-hydroclimatic-data-accessing-with-hyriver-for-python/)

The script ['retrieve_basin_data.ipynb'](https://github.com/TrevorJA/Folium_Interactive_Map_Demo/blob/main/retrieve_basin_data.ipynb) is set up to retrieve several features from the basin, including:
- Basin boundary
- Mainstem river
- Tributary rivers
- USGS gauge locations
- New Mexico Water Data Initiative (NMWDI) Sites
- HUC12 pour points

The geospatial data (longitude and latitudes) for each of these features are exported to [`data/basin_data.csv`](https://github.com/TrevorJA/Folium_Interactive_Map_Demo/blob/main/data/basin_data.csv) and used later to generate the folium map.

### Constructing a folium hydrologic map

Like many other data visualization programs, `folium` maps are constructed by iteratively adding features or map elements to the main map object. It is easy to add a map feature, however it takes more care to ensure that the features are well-organized and respond to user interaction the way you want them to.

In this demo, I deconstruct the process into five parts:
1. Initializing the map
2. Initializing feature groups (layers)
3.  Adding points to feature layers
4. Adding polygons & lines to feature layers
5. Adding layers onto the map

Before going any further, we need to import the necessary packages, load our basin data, and convert the geospatial data to `geopandas.GeoDataFrame()` geometry objects:


In [1]:
# Import packages
import pandas as pd
import folium
from folium.plugins import MarkerCluster
import geopandas as gpd
from shapely.geometry import Point, LineString
from shapely import wkt

# Specify the coordinate reference system (CRS)
crs = 4386

In [2]:
# Load the basin data
basin_data = pd.read_csv("basin_data.csv", sep = ',', index_col=0)

# Convert to a geoDataFrame and geometry objects
basin_data['geometry'] = basin_data['geometry'].apply(wkt.loads)
geodata = gpd.GeoDataFrame(basin_data, crs = crs)
geodata.head(2)

,geometry,type,description
0,"POLYGON ((-111.69395 41.60240, -111.70449 41.5...",basin,Basin boundary.
1,"LINESTRING (-111.71959 41.62961, -111.72027 41...",main,The mainflow river upstream of USGS-10113500


In [3]:
basin_data

,geometry,type,description
0,"POLYGON ((-111.69395 41.60240, -111.70449 41.5...",basin,Basin boundary.
1,"LINESTRING (-111.71959 41.62961, -111.72027 41...",main,The mainflow river upstream of USGS-10113500
2,"LINESTRING (-111.71435 41.62877, -111.71532 41...",main,The mainflow river upstream of USGS-10113500
3,"LINESTRING (-111.71340 41.62909, -111.71334 41...",main,The mainflow river upstream of USGS-10113500
4,"LINESTRING (-111.70813 41.63044, -111.70865 41...",main,The mainflow river upstream of USGS-10113500
...,...,...,...
235,POINT (-111.55776 41.57115),pourpoint,HUC12 Pour Point
236,POINT (-111.55782 41.57141),pourpoint,HUC12 Pour Point
237,POINT (-111.70787 41.63072),pourpoint,HUC12 Pour Point
238,POINT (-111.52388 41.68777),pourpoint,HUC12 Pour Point


Additionally, I start by specifying a couple design options before getting started (colors, line widths, etc.). I do this at the start so that I can easily test different colors, and I store all of these specifications in a `dict`. This step is not necessary, but just helps to stay organized. The following code block shows _some_ of these specifications, but some are left out just to make this post shorter:

In [4]:
## Plot options
# Line widths
basin_linewidth = 2
mainstem_linewidth = 3
tributary_linewidth = 1

# Line colors
mainstem_color = '#0a4b6e'
tributary_color = '#5e9eab'
basin_color = '#213b4a'

# Point colors
usgs_color = '#2a6339'
# nmwdi_color = '#70207a'
pourpoint_color = '#8f5e1a'

# Point sizes
usgs_size = 10
# nmwdi_size = 10
pourpoint_size = 5

# Other options
fill_opacity = 0.85
popup_width = 300

# Combine options into a dictionary
plot_options = {
    'station':{'color': usgs_color,
               'size': usgs_size},
    'pourpoint': {'color': pourpoint_color,
                  'size': pourpoint_size},
    # 'nmwdi': {'color': nmwdi_color, 
    #           'size': nmwdi_size}
    }

#### Initializing the map

We start to construct our map using the `folium.Map()` function:

The `location` and `zoom_start` arguments set the default view; the user will be able to pan and zoom around the map, but this will be the starting location.

The `tile` argument in the initial `folium.Map()` calls sets the default base-map style, but different styles can be added using the `folium.TileLayer()`  function. Each `TileLayer()`  call adds a different base-map style that is then available from the drop-down menu in the final figure.

In [5]:
# Initialize the map
start_coords = [41.5, -111.18]  
geomap = folium.Map(location = start_coords, 
                    zoom_start = 9.2,
                    tiles = 'cartodbpositron',
                    control_scale = True)

## Add different tiles (base maps)
folium.TileLayer('openstreetmap').add_to(geomap)
folium.TileLayer('stamenwatercolor').add_to(geomap)
folium.TileLayer('stamenterrain').add_to(geomap)


#### Initializing feature groups (layers)

Before we add any lines or makers to the map, it is best to set up different layers using `folium.FeatureGroup()`. When the map is complete, the different feature groups can be toggled on-and-off using the drop-down menu.

Below, I initialize a `folium.FeatureGroup` for each of the six feature types in the map.

In [6]:
# Start feature groups for toggle functionality
basin_layer = folium.FeatureGroup(name = 'Basin Boundary', show=True)
usgs_layer = folium.FeatureGroup(name = 'USGS Gauges', show=True)
mainstem_layer = folium.FeatureGroup(name = 'Mainstem River', show=True)
tributary_layer = folium.FeatureGroup(name='Tributary Rivers', show=False)
pourpoint_layer = folium.FeatureGroup(name= 'HUC12 Pour Points', show=False)
# nmwdi_layer = folium.FeatureGroup(name='NM Water Data Initiative Gauge', show=True)

#### Adding polygons & lines to feature layers

I've found that it can be difficult to add Polygons or Lines to a `folium` map if the coordinate geometry are not formatted correctly. For me, the best method has been to convert the polygon or line data to a `geopandas.GeoSeries` and then converting this to JSON format data using the `.to_json()` method.

Once in JSON format, the data can be added to the map using the `folium.GeoJson()` method similar to other features.  Although, rather than adding it to the map directly, we add it to the specific feature layer so that we can toggle things later.

Below, I show how I add the basin boundary polygon to the map. This needs to be repeated for the mainstem and tributary river lines, and the full code is included in [`folium_map_demo.ipynb`](https://github.com/TrevorJA/Folium_Interactive_Map_Demo/blob/main/folium_map_demo.ipynb). 

In [7]:
## Plotting Polygons and Lines

# Plot basin border
for i,r in geodata.loc[geodata['type'] == 'basin'].iterrows():
    # Convert the Polygon or LineString to geoJSON format
    geo_json = gpd.GeoSeries(r['geometry']).simplify(tolerance = 0.000001).to_json()
    geo_json = folium.GeoJson(data= geo_json,
                              style_function=lambda x: {'fillcolor': 'none',
                                                        'weight': basin_linewidth, 
                                                        'color': basin_color,
                                                        'opacity': 1,
                                                        'fill_opacity': 1,
                                                        'fill': False})
    # Add popup with line description
    folium.Popup(r.description,
                 min_width = popup_width,
                 max_width= popup_width).add_to(geo_json)
    
    # Add the feature to the appropriate layer
    geo_json.add_to(basin_layer)
    
# Add mainstem flowlines
for i,r in geodata.loc[geodata['type'] == 'main'].iterrows():
    
    geo_json = gpd.GeoSeries(r['geometry']).simplify(tolerance = 0.000001).to_json()
    geo_json = folium.GeoJson(data= geo_json,
                              style_function=lambda x: {'fillcolor': 'none',
                                                        'weight': mainstem_linewidth, 
                                                        'color': mainstem_color,
                                                        'opacity': 1,
                                                        'fill_opacity': 1,
                                                        'fill': False})
    folium.Popup(r.description,
                min_width = popup_width,
                max_width= popup_width).add_to(geo_json)
    geo_json.add_to(mainstem_layer)
    
    
# Add tributary flow lines
for i,r in geodata.loc[geodata['type'] == 'tributary'].iterrows():
    
    geo_json = gpd.GeoSeries(r['geometry']).simplify(tolerance = 0.000001).to_json()
    geo_json = folium.GeoJson(data= geo_json,
                              style_function=lambda x: {'fillcolor': 'none',
                                                        'weight': tributary_linewidth, 
                                                        'color': tributary_color,
                                                        'opacity': 1,
                                                        'fill_opacity': 1,
                                                        'fill': False})
    folium.Popup(r.description,
                min_width = popup_width,
                max_width= popup_width).add_to(geo_json)
    geo_json.add_to(tributary_layer)

#### Adding points to feature layers

The `folium.CircleMarker()` is used to add circle points using a specific coordinate location.

The following code shows how I am iteratively adding different points to the different layers of the map based upon the feature type.  

For each point, I extract the latitude and longitude (`coords = [point.geometry.y, point.geometry.x]`) and pass it to the `folium.CircleMarker()` function with colors and sizes specific to each of the three different point features (USGS gauge stations (`station`), NMWDI (`nmwdi`), and HUC12 pourpoints (`pourpoint`)):

In [8]:
## Plotting point data

plot_types = ['station', 'pourpoint']
plot_layers = [usgs_layer, pourpoint_layer]

for i, feature_type in enumerate(plot_types):

    map_layer = plot_layers[i]
    
    for _, point in geodata.loc[geodata['type'] == feature_type].iterrows():
        
        coords = [point.geometry.y, point.geometry.x]
        
        textbox = folium.Popup(point.description,
                               min_width= popup_width,
                               max_width= popup_width)
 
        folium.CircleMarker(coords,
                            popup= textbox,
                            fill_color = plot_options[feature_type]['color'],
                            fill = True,
                            fill_opacity = fill_opacity,
                            radius= plot_options[feature_type]['size'],
                            color = plot_options[feature_type]['color']).add_to(map_layer)
 


#### Last step: adding layers onto map

Now, if you try to visualize the map it will be empty! This is because we have not added the feature layers to the map.  In this last step, we add each of the six feature layers to the map and also add the `folium.LayerControl()` which will allow for us to toggle the different layers on-and-off:

In [9]:
# Add all feature layers to the map
basin_layer.add_to(geomap)
usgs_layer.add_to(geomap)
mainstem_layer.add_to(geomap)
tributary_layer.add_to(geomap)
pourpoint_layer.add_to(geomap)
# nmwdi_layer.add_to(geomap)

# Add the toggle option for layers
folium.LayerControl().add_to(geomap)

#### Viewing, saving, and sharing the map

Viewing your map is as easy as calling the map name at any point in the script (i.e., `geomap`), and `folium` makes it easy to save the map as an HTML using the `map.save()` function as shown here:

In [10]:
# Save and view the map
geomap.save("BSF_basin_map.html")
geomap

Thanks for viewing!